In [461]:
def fast_pow(c, N, d):
    d_bin = "{0:b}".format(d)
    d_len = len(d_bin)
    reductions = 0
    h = 0
    x = c

    for j in range(1, d_len):
        x, r = mod_reduce(x ** 2, N)
        reductions = reductions + r
        if d_bin[j] == "1":
            x, r = mod_reduce(x * c, N)
            reductions = reductions + r
            h = h + 1

    return x, h, reductions

def mod_reduce(a, b):
    reductions = 0
    if a >= b:
        a = a % b
        reductions = 1
    return a, reductions


In [462]:
from sympy import randprime, mod_inverse
import math
import numpy as np

def GenModulus(w):
    n = len(w) // 2
    p = randprime(2 ** n, 2 ** (n+1))
    q = randprime(2 ** n, 2 ** (n+1))
    N = p * q
    return N, p, q

def GenRSA(w):
    N, p, q = GenModulus(w)
    m = (p-1) * (q-1)
    e = 2 ** 16 + 1
    d = mod_inverse(e, m)
    return N, e, d, p, q

def enc(x, N, e):
    return fast_pow(x, N, e) #x ** e % N

def dec(c, N, d):
    return fast_pow(c, N, d) #c ** d % N

In [463]:
w = "11111111"
N, e, d, p, q = GenRSA(w)

print(N, e, d, p, q)

899 65537 593 31 29


In [464]:
d_bin = "{0:b}".format(d)
d_bin

'1001010001'

In [465]:
key = '1'
for b in range(1, len("{0:b}".format(d))):
    res0 = []
    for c in range(2, N * 2):
        d_g = int(key + '01')
        x1, h1, r1 = fast_pow(c, N, d)
        x2, h2, r2 = fast_pow(c, N, d_g)
        res0.append((r1, r2))

    res1 = []
    for c in range(2, N * 2):
        d_g = int(key + '10')
        x1, h1, r1 = fast_pow(c, N, d)
        x2, h2, r2 = fast_pow(c, N, d_g)
        res1.append((r1, r2))
    
    key += '0' if np.var(res0) < np.var(res1) else '1' 

key

'1111111111'

## Task 2 (Blind signatures)

In [466]:
import hashlib
import struct
import random
import sympy

def fastpow(x, N, e):
    return fast_pow(x, N, e)[0]

def h(num):
    return int(hashlib.sha256(struct.pack('<I', num)).hexdigest()[:6], 16)

def sign(m, N, d):
    return fastpow(m, N, d)

def verify(m, s, N, e):
    hm = h(m) % N
    return hm == fastpow(s, N, e)

In [467]:
N, e, d, p, q = GenRSA("11111111")
N, e, d

(551, 65537, 89)

In [468]:
m = 100

r =  random.randint(3, N)
r_inv = mod_inverse(r, N)

m_prime = m * fast_pow(r, N, e)[0] % N

s_real = sign(m, N, d) # m ^ d % N
s_r = sign(m_prime, N, d) * r_inv % N # (m * (r ^ e)) ^ d

s_real == s_r

True

In [469]:
dec(enc(2, N, e)[0], N, d)[0]

2